In [1]:
import pandas as pd
from itertools import combinations
from typing import List, Dict, Tuple, Any

In [2]:
def normalize_columns(df: pd.DataFrame, freq_col: str = 'purchase_frequency', rec_col: str = 'recency') -> pd.DataFrame:
    """
    Normalize frequency and recency using min-max normalization:
    normalized_value = (value - min) / (max - min)
    Adds two new columns: 'normalized_f' and 'normalized_r'.
    
    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data to normalize.
    freq_col : str, optional
        The name of the column representing purchase frequency. Default is 'purchase_frequency'.
    rec_col : str, optional
        The name of the column representing recency. Default is 'recency'.
        
    Returns:
    -------
    pd.DataFrame
        The DataFrame with two new normalized columns added.
    """
    for col, norm_col in [(freq_col, 'normalized_f'), (rec_col, 'normalized_r')]:
        min_val = df[col].min()  # Find the minimum value in the column
        max_val = df[col].max()  # Find the maximum value in the column
        if max_val > min_val:
            # Apply min-max normalization
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            # Handle case where all values are identical to avoid division by zero
            df[norm_col] = 1.0
    return df

In [3]:
def calculate_normalized_score(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the normalized_score for each product as the sum of normalized frequency and normalized recency.
    
    Parameters:
    ----------
    df : pd.DataFrame
        The DataFrame containing normalized frequency ('normalized_f') and recency ('normalized_r').
        
    Returns:
    -------
    pd.DataFrame
        The DataFrame with a new column 'normalized_score' added.
    """
    df['normalized_score'] = df['normalized_f'] + df['normalized_r']
    return df

In [4]:
def calculate_business_score(subset: Tuple[Tuple[Any, ...], ...]) -> float:
    """
    Calculate a business-specific score for a given subset of products.
    Customize this logic as per business requirements.
    
    Parameters:
    ----------
    subset : Tuple of Tuples
        A subset of products, where each product is represented as a tuple.
        
    Returns:
    -------
    float
        The business-specific score for the subset.
    """
    # Example placeholder: assign a constant value
    # TODO: Replace with actual business logic
    return 2.0

In [5]:
def is_category_unique(subset: Tuple[Tuple[Any, ...], ...]) -> bool:
    """
    Check if all categories in the subset are unique.
    
    Parameters:
    ----------
    subset : Tuple of Tuples
        A subset of products.
        
    Returns:
    -------
    bool
        True if all categories are unique, False otherwise.
    """
    categories = [p[1] for p in subset]  # Extract the category from each product
    return len(set(categories)) == len(categories)  # Check for uniqueness

In [6]:
def is_within_budget(subset: Tuple[Tuple[Any, ...], ...], budget: float) -> bool:
    """
    Check if the sum of prices in the subset does not exceed the budget.
    
    Parameters:
    ----------
    subset : Tuple of Tuples
        A subset of products.
    budget : float
        The maximum allowed total price for the subset.
        
    Returns:
    -------
    bool
        True if total price is within budget, False otherwise.
    """
    total_price = sum(p[2] for p in subset)  # Sum the prices of all products in the subset
    return total_price <= budget

In [7]:
def calculate_final_score(subset: Tuple[Tuple[Any, ...], ...]) -> float:
    """
    Calculate the final score for a given subset of products.
    final_score = sum of normalized_score + business_score
    
    Parameters:
    ----------
    subset : Tuple of Tuples
        A subset of products.
        
    Returns:
    -------
    float
        The final score for the subset.
    """
    base_score = sum(p[3] for p in subset)  # Sum of normalized scores (assuming p[3] is normalized_score)
    business = calculate_business_score(subset)  # Business-specific score
    return base_score + business


In [8]:
def generate_all_valid_bundles(
    products: List[Tuple[int, str, float, float]],
    budget: float,
    min_bundle_size: int = 2
) -> List[Dict]:
    """
    Generate all valid bundles of products that:
    - Have at least min_bundle_size products
    - Have unique categories
    - Are within the budget
    
    Parameters:
    ----------
    products : List of Tuples
        Each product is represented as a tuple: (product_id, category, price, normalized_score).
    budget : float
        The maximum allowed total price for a bundle.
    min_bundle_size : int, optional
        The minimum number of products required in a bundle. Default is 2.
        
    Returns:
    -------
    List of Dicts
        Each dictionary represents a valid bundle with its products, total price, and score.
    """
    valid_bundles = []
    n = len(products)
    for r in range(min_bundle_size, n + 1):
        # Generate all possible combinations of size r
        for subset in combinations(products, r):
            if is_category_unique(subset) and is_within_budget(subset, budget):
                final_score = calculate_final_score(subset)
                bundle_price = sum(p[2] for p in subset)  # Total price of the bundle
                valid_bundles.append({
                    'bundle': subset,  # The subset of products
                    'price': bundle_price,
                    'score': final_score
                })
    return valid_bundles

In [9]:
def is_extendable(
    bundle: Dict,
    products: List[Tuple[int, str, float, float]],
    budget: float
) -> bool:
    """
    Check if a given bundle can be extended by adding another product without violating constraints.
    
    Parameters:
    ----------
    bundle : Dict
        The current bundle with keys 'bundle', 'price', and 'score'.
    products : List of Tuples
        All available products.
    budget : float
        The maximum allowed total price for a bundle.
        
    Returns:
    -------
    bool
        True if the bundle can be extended, False otherwise.
    """
    current_ids = {p[0] for p in bundle['bundle']}  # Current product IDs in the bundle
    current_categories = {p[1] for p in bundle['bundle']}  # Current categories in the bundle
    current_price = bundle['price']  # Current total price
    
    for p in products:
        if p[0] not in current_ids:
            # Check if adding this product maintains unique categories and stays within budget
            if p[1] not in current_categories and (current_price + p[2] <= budget):
                return True  # Bundle can be extended
    return False  # No valid extensions found

In [10]:
def generate_maximal_bundles(
    products: List[Tuple[int, str, float, float]],
    budget: float,
    min_bundle_size: int = 2
) -> List[Dict]:
    """
    Generate only maximal bundles:
    A maximal bundle is one that cannot be extended further without violating constraints.
    
    Parameters:
    ----------
    products : List of Tuples
        Each product is represented as a tuple: (product_id, category, price, normalized_score).
    budget : float
        The maximum allowed total price for a bundle.
    min_bundle_size : int, optional
        The minimum number of products required in a bundle. Default is 2.
        
    Returns:
    -------
    List of Dicts
        Each dictionary represents a maximal bundle with its products, total price, and score.
    """
    all_bundles = generate_all_valid_bundles(products, budget, min_bundle_size)
    maximal_bundles = []
    for b in all_bundles:
        if not is_extendable(b, products, budget):
            maximal_bundles.append(b)  # Only add if the bundle cannot be extended
    return maximal_bundles

In [11]:
def build_bundle_rows_for_consultant(
    group: pd.DataFrame,
    consultant_id: Any,
    maximal_bundles: List[Dict]
) -> List[Dict]:
    """
    Convert maximal bundles into rows. Each row represents one bundle:
    - consultant_id
    - products: list of product_ids in the bundle
    - price: total price of the bundle
    - total_normalized_score: sum of normalized_score of products in the bundle
    - bundle_score: final score of the bundle
    
    Parameters:
    ----------
    group : pd.DataFrame
        The DataFrame group corresponding to a single consultant.
    consultant_id : Any
        The identifier for the consultant.
    maximal_bundles : List of Dicts
        The list of maximal bundles for this consultant.
        
    Returns:
    -------
    List of Dicts
        Each dictionary represents a bundle row ready for output.
    """
    results = []
    for mb in maximal_bundles:
        product_ids = [p[0] for p in mb['bundle']]  # Extract product IDs
        # Extract product information from the group DataFrame
        subset_df = group[group['product_id'].isin(product_ids)]
        
        total_normalized_score = subset_df['normalized_score'].sum()  # Sum of normalized scores
        # The mb['score'] is already the final_score = total_normalized_score + business_score
        
        result_row = {
            'consultant_id': consultant_id,
            'products': product_ids,
            'price': mb['price'],
            'total_normalized_score': total_normalized_score,
            'bundle_score': mb['score']
        }
        results.append(result_row)
    return results

In [12]:

def process_csv(
    input_csv: str,
    output_csv: str,
    budget: float,
    min_bundle_size: int = 2,
    max_rows: int = None
) -> None:
    """
    Main function to:
    - Load data from input CSV
    - Normalize frequency and recency
    - Compute normalized_score
    - Generate maximal bundles per consultant
    - Output results to a CSV (one row per bundle)
    
    Parameters
    ----------
    input_csv : str
        Path to the input CSV file.
    output_csv : str
        Path where the output CSV file should be written.
    budget : float
        The purchasing power or maximum allowed total price for a bundle.
    min_bundle_size : int, optional
        The minimum number of products required in a bundle. Default is 2.
    max_rows : int, optional
        If given, read only the specified number of rows from the input file.
    """
    # Load input CSV into a DataFrame
    df = pd.read_csv(input_csv)
    if max_rows is not None:
        df = df.head(max_rows)  # Limit the number of rows if max_rows is specified
    
    # Normalize frequency and recency columns
    df = normalize_columns(df, freq_col='purchase_frequency', rec_col='recency')
    
    # Calculate normalized_score for each product
    df = calculate_normalized_score(df)
    
    # Initialize a list to hold all result rows
    results = []
    
    # Process each consultant separately by grouping the DataFrame
    for consultant_id, group in df.groupby('consultant_id'):
        # Convert relevant product information into a list of tuples
        # Each tuple contains: (product_id, category, price, normalized_score)
        products = list(group[['product_id', 'category', 'price', 'normalized_score']].itertuples(index=False, name=None))
        
        # Generate maximal bundles for this consultant based on constraints
        maximal = generate_maximal_bundles(products, budget, min_bundle_size)
        
        # Convert each maximal bundle into a result row
        consultant_results = build_bundle_rows_for_consultant(group, consultant_id, maximal)
        results.extend(consultant_results)  # Add to the overall results
    
    # Convert the list of result dictionaries into a DataFrame
    results_df = pd.DataFrame(results)
    
    # Optional: Sort the results for better readability or specific analysis
    # For example, sort by consultant_id and then by bundle_score in descending order
    results_df.sort_values(by=['consultant_id', 'bundle_score'], inplace=True, ascending=[True, False])
    
    # Write the final results to the specified output CSV file
    results_df.to_csv(output_csv, index=False)


In [13]:

process_csv(
    input_csv='first_500_rows.csv',
    output_csv='first_500_rows_res.csv',
    budget=200,            # Set the budget constraint for bundles
    min_bundle_size=2,     # Set the minimum number of products in a bundle
    max_rows=500           # Optionally limit the number of rows processed
)

KeyError: 'purchase_frequency'

In [ ]:
P1  C1
P2  C2
P3  C1
P4  C4
P5  C5
P6  C6

p1, p2, P4
p1, p2, p5

category, sub category, type, sub type - type should not be same

In [ ]:
import pandas as pd

# Load the CSV file (replace 'input_file.csv' with the path to your input file)
input_file = 'F1_test.csv'  # Replace with your file path
output_file = 'first_500_rows.csv'  # Path for the output file

# Read the input CSV
data = pd.read_csv(input_file)

# Select the first 500 rows
first_500_rows = data.head(500)

# Save the first 500 rows to a new CSV file
first_500_rows.to_csv(output_file, index=False)

print(f"The first 500 rows have been saved to {output_file}.")


The first 500 rows have been saved to first_500_rows.csv.


### More optimized approach


In [ ]:
# import pandas as pd
# from itertools import combinations
# from typing import List, Dict, Tuple, Any

# def normalize_columns(df: pd.DataFrame, freq_col: str = 'purchase_frequency', rec_col: str = 'recency') -> pd.DataFrame:
#     """
#     Normalize the frequency and recency columns using min-max normalization.
#     normalized_value = (value - min) / (max - min)
#     If all values are the same (min == max), normalization defaults them all to 1.0.

#     Parameters
#     ----------
#     df : pd.DataFrame
#         The input DataFrame containing frequency and recency columns.
#     freq_col : str
#         The column name representing purchase frequency.
#     rec_col : str
#         The column name representing recency.

#     Returns
#     -------
#     pd.DataFrame
#         The DataFrame with two new columns: 'normalized_f' and 'normalized_r'.
#     """
#     for col, norm_col in [(freq_col, 'normalized_f'), (rec_col, 'normalized_r')]:
#         min_val = df[col].min()
#         max_val = df[col].max()
#         if max_val > min_val:
#             df[norm_col] = (df[col] - min_val) / (max_val - min_val)
#         else:
#             # If there's no range (min == max), set all values to 1 to avoid division by zero
#             df[norm_col] = 1.0
#     return df


# def calculate_normalized_score(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Calculate a 'normalized_score' as the sum of normalized frequency and normalized recency.

#     Parameters
#     ----------
#     df : pd.DataFrame
#         The input DataFrame containing 'normalized_f' and 'normalized_r' columns.

#     Returns
#     -------
#     pd.DataFrame
#         The DataFrame with an additional 'normalized_score' column.
#     """
#     df['normalized_score'] = df['normalized_f'] + df['normalized_r']
#     return df


# def calculate_business_score(subset: Tuple[Tuple[Any, ...], ...]) -> float:
#     """
#     Calculate a business-specific score for a given subset of products.
#     This is a placeholder function. Modify this logic per business requirements.

#     Parameters
#     ----------
#     subset : tuple of tuples
#         Each element is a product represented by (product_id, category, price, normalized_score).

#     Returns
#     -------
#     float
#         The business-specific score. Currently returns a constant value for demonstration.
#     """
#     return 2.0


# def generate_all_valid_bundles(
#     products: List[Tuple[int, str, float, float]],
#     budget: float,
#     min_bundle_size: int = 2
# ) -> List[Dict]:
#     """
#     Generate all valid bundles of products that:
#     - Have at least `min_bundle_size` products.
#     - Contain unique categories (no category repeated).
#     - Are within the given budget.

#     This function is optimized compared to a naive approach:
#     - We use a category-to-integer mapping and a bitmask to quickly check if a category is unique.
#     - We integrate the checks for budget and category uniqueness directly as we build the subset.

#     Parameters
#     ----------
#     products : list of tuples
#         Each tuple: (product_id, category, price, normalized_score)
#     budget : float
#         The maximum allowed total price for a bundle.
#     min_bundle_size : int
#         The minimum number of products in a valid bundle.

#     Returns
#     -------
#     list of dict
#         Each dict has keys: 'bundle', 'price', 'score'.
#         'bundle' is a tuple of product tuples, 'price' is the total bundle price, 'score' is the final score.
#     """
#     # Map each unique category to an integer ID for bitmask operations
#     category_to_id = {}
#     cat_id = 0
#     for _, cat, _, _ in products:
#         if cat not in category_to_id:
#             category_to_id[cat] = cat_id
#             cat_id += 1

#     valid_bundles = []
#     n = len(products)

#     # Generate combinations of products of size r, for r from min_bundle_size to n
#     for r in range(min_bundle_size, n + 1):
#         for subset in combinations(products, r):
#             category_mask = 0  # bitmask to track used categories
#             total_price = 0.0
#             total_normalized_score = 0.0
#             valid = True  # flag to indicate if this subset is valid

#             # Check each product in the subset
#             for p in subset:
#                 # p is (product_id, category, price, normalized_score)
#                 cid = category_to_id[p[1]]  # get the integer ID of the category
#                 # Check if this category is already used by checking the category_mask bit
#                 if (category_mask & (1 << cid)) != 0:
#                     # Category already used in this subset; not valid
#                     valid = False
#                     break
#                 category_mask |= (1 << cid)  # mark this category as used

#                 # Accumulate price and check against budget
#                 total_price += p[2]
#                 if total_price > budget:
#                     # Over the allowed budget; no need to continue
#                     valid = False
#                     break

#                 # Accumulate the total normalized score
#                 total_normalized_score += p[3]

#             if not valid:
#                 # This subset fails either category uniqueness or budget constraints
#                 continue

#             # If we reach here, the subset is valid
#             # Calculate the final score (normalized_score sum + business_score)
#             business = calculate_business_score(subset)
#             final_score = total_normalized_score + business

#             valid_bundles.append({
#                 'bundle': subset,
#                 'price': total_price,
#                 'score': final_score
#             })

#     return valid_bundles


# def is_extendable(
#     bundle: Dict,
#     products: List[Tuple[int, str, float, float]],
#     budget: float
# ) -> bool:
#     """
#     Check if a given bundle can be extended by adding another product without violating constraints:
#     - Adding another product must not break category uniqueness.
#     - Adding another product must remain within the budget.

#     Parameters
#     ----------
#     bundle : dict
#         A dictionary representing a current bundle with keys: 'bundle', 'price', 'score'.
#     products : list of tuples
#         All available products (product_id, category, price, normalized_score).
#     budget : float
#         Maximum allowed total price for an extended bundle.

#     Returns
#     -------
#     bool
#         True if there is at least one product that can be added without violating constraints, False otherwise.
#     """
#     # Extract current bundle info for quick checks
#     current_ids = {p[0] for p in bundle['bundle']}
#     current_categories = {p[1] for p in bundle['bundle']}
#     current_price = bundle['price']

#     # Try to find any product not in the bundle and not sharing a category, and fits the budget
#     for p in products:
#         if p[0] not in current_ids:
#             if p[1] not in current_categories and (current_price + p[2] <= budget):
#                 return True
#     return False


# def generate_maximal_bundles(
#     products: List[Tuple[int, str, float, float]],
#     budget: float,
#     min_bundle_size: int = 2
# ) -> List[Dict]:
#     """
#     Generate only maximal bundles. A maximal bundle is one that cannot be extended further
#     without violating the category uniqueness or budget constraints.

#     Parameters
#     ----------
#     products : list of tuples
#         (product_id, category, price, normalized_score)
#     budget : float
#         Maximum budget for a bundle.
#     min_bundle_size : int
#         Minimum number of products in a bundle.

#     Returns
#     -------
#     list of dict
#         Each dict represents a maximal bundle with keys: 'bundle', 'price', 'score'.
#     """
#     all_bundles = generate_all_valid_bundles(products, budget, min_bundle_size)
#     maximal_bundles = []
#     for b in all_bundles:
#         # A bundle is maximal if it cannot be extended
#         if not is_extendable(b, products, budget):
#             maximal_bundles.append(b)
#     return maximal_bundles


# def build_bundle_rows_for_consultant(
#     group: pd.DataFrame,
#     consultant_id: Any,
#     maximal_bundles: List[Dict]
# ) -> List[Dict]:
#     """
#     Convert maximal bundle information into output rows for a given consultant.
#     Each row includes:
#     - consultant_id
#     - a list of product_ids in the bundle
#     - the total price of the bundle
#     - the total normalized score of products in the bundle
#     - the final bundle score (which includes business logic)

#     Parameters
#     ----------
#     group : pd.DataFrame
#         The consultant-specific subset of the DataFrame.
#     consultant_id : Any
#         The unique identifier of the consultant.
#     maximal_bundles : list of dict
#         The maximal bundles for this consultant.

#     Returns
#     -------
#     list of dict
#         Each dict represents one row with keys:
#         'consultant_id', 'products', 'price', 'total_normalized_score', 'bundle_score'.
#     """
#     results = []
#     for mb in maximal_bundles:
#         product_ids = [p[0] for p in mb['bundle']]
#         # subset_df filters only the products in the current bundle
#         subset_df = group[group['product_id'].isin(product_ids)]

#         # total_normalized_score should match the sum of p[3] for the products in the bundle
#         total_normalized_score = subset_df['normalized_score'].sum()

#         # mb['score'] = total_normalized_score + business_score
#         result_row = {
#             'consultant_id': consultant_id,
#             'products': product_ids,
#             'price': mb['price'],
#             'total_normalized_score': total_normalized_score,
#             'bundle_score': mb['score']
#         }
#         results.append(result_row)
#     return results


# def process_csv(
#     input_csv: str,
#     output_csv: str,
#     budget: float,
#     min_bundle_size: int = 2,
#     max_rows: int = None
# ) -> None:
#     """
#     Main function to:
#     - Load data from input CSV
#     - Normalize frequency and recency columns
#     - Compute normalized_score
#     - Generate maximal bundles per consultant
#     - Write results to an output CSV

#     Parameters
#     ----------
#     input_csv : str
#         Path to the input CSV file.
#     output_csv : str
#         Path where the output CSV file should be written.
#     budget : float
#         The maximum allowed total price for a bundle.
#     min_bundle_size : int
#         The minimum number of products required in each bundle.
#     max_rows : int, optional
#         If specified, read only the first max_rows of the input CSV.
#     """
#     # Load the input CSV
#     df = pd.read_csv(input_csv)
#     if max_rows is not None:
#         df = df.head(max_rows)
    
#     # Normalize the necessary columns
#     df = normalize_columns(df, freq_col='purchase_frequency', rec_col='recency')
    
#     # Compute the normalized_score for each product
#     df = calculate_normalized_score(df)
    
#     # We will gather results for all consultants
#     results = []
    
#     # Process each consultant separately
#     for consultant_id, group in df.groupby('consultant_id'):
#         # Convert product data to tuples for easier manipulation
#         products = list(group[['product_id', 'category', 'price', 'normalized_score']].itertuples(index=False, name=None))
        
#         # Generate maximal bundles for this consultant
#         maximal = generate_maximal_bundles(products, budget, min_bundle_size)
        
#         # Convert these bundles into row dictionaries for output
#         consultant_results = build_bundle_rows_for_consultant(group, consultant_id, maximal)
#         results.extend(consultant_results)
    
#     # Create a DataFrame from all results
#     results_df = pd.DataFrame(results)
    
#     # Sort results by consultant_id, and within that by bundle_score (descending)
#     results_df.sort_values(by=['consultant_id', 'bundle_score'], inplace=True, ascending=[True, False])
    
#     # Write the results to the output CSV
#     results_df.to_csv(output_csv, index=False)


# if __name__ == "__main__":
#     # Example usage (replace file paths as needed):
#     process_csv('data/F1_test.csv', 'data/F1_test_res.csv', budget=200, min_bundle_size=2, max_rows=500)
